In [1]:
import nltk
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def sentence(record):
    try:
        split_record = record.strip().split('\t')
        sentence_1 = split_record[8]
        sentence_2 = split_record[9]
        return (sentence_1,sentence_2)
    except:
        return()

def wordMatchNltk(record):
    sentence = record[0]+record[1]
    tokens = word_tokenize(sentence)
    words = [word for word in tokens if word.isalpha()]
    return [word.lower() for word in words]


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1589977564409_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Read file from storage
devMatched = sc.textFile('s3://bgao6342/Assignment/dev_matched.tsv')
devMismatched = sc.textFile('s3://bgao6342/Assignment/dev_mismatched.tsv')
testMatched = sc.textFile('s3://bgao6342/Assignment/test_matched.tsv')
testMismatched = sc.textFile('s3://bgao6342/Assignment/test_mismatched.tsv')

#extract sentences and remove headers
# for devMatched.tsv
devMatched_selected = devMatched.map(sentence)
header = devMatched_selected.first()
devMatched_noheader = devMatched_selected.filter(lambda x: x != header)

# for devMismatched.tsv
devMismatched_selected = devMismatched.map(sentence)
header = devMismatched_selected.first()
devMismatched_noheader = devMismatched_selected.filter(lambda x: x != header)

# for testMatched.tsv
testMatched_selected = testMatched.map(sentence)
header = testMatched_selected.first()
testMatched_noheader = testMatched_selected.filter(lambda x: x != header)

# for testMismatched.tsv
testMismatched_selected = testMismatched.map(sentence)
header = testMismatched_selected.first()
testMismatched_noheader = testMismatched_selected.filter(lambda x: x != header)

# Extract words from sentences 
word_devMatched = devMatched_noheader.flatMap(wordMatchNltk)
word_devMismatched = devMismatched_noheader.flatMap(wordMatchNltk)
word_testMatched = testMatched_noheader.flatMap(wordMatchNltk)
word_testMismatched = testMismatched_noheader.flatMap(wordMatchNltk)

# Union data 
word_matched = sc.union([word_devMatched, word_testMatched])
word_mismatched = sc.union([word_devMismatched, word_testMismatched])

#repartition and count the number of distinct of words
repartition_matched = word_matched.repartition(8)
repartition_mismatched = word_mismatched.repartition(8)
distinct_word_matched = repartition_matched.distinct().persist()
distinct_word_mismatched = repartition_mismatched.distinct().persist()

#count words
commonWords = distinct_word_matched.intersection(distinct_word_mismatched)
commonWordsNumber = commonWords.count()
matchedWordsNumber = distinct_word_matched.count()
mismatchedWordsNumber = distinct_word_mismatched.count()
print('The number of common words between matched and mismatched sets:')
print(commonWordsNumber)
print('The number of words unique to the matched sets:')
print(matchedWordsNumber-commonWordsNumber)
print('The number of words unique to the mismatched sets:')
print(mismatchedWordsNumber-commonWordsNumber)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The number of common words between matched and mismatched sets:
8927
The number of words unique to the matched sets:
11134
The number of words unique to the mismatched sets:
6466